In [4]:
import os
import besos
from besos import sampling
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist, DependentParameter, CategoryParameter
from besos.objectives import MeterReader,time_series_values,EPResults
from besos.optimizer import NSGAII, df_solution_to_solutions
import pandas as pd

In [8]:
# Open the IDF file
idf_1 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/1_layer_window.idf"
idf_2 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/2_layer_window.idf"
idf_3 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/3_layer_window.idf"
idfs = [idf_1,idf_2,idf_3]
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'
building = ef.get_building(idf_1)

In [12]:
# check the version if 8.X then convert it using E+
ep.get_idf_version(building)

'9.0'

In [11]:
# get a list of the Name property of all IDF objects of class MATERIAL
'''
for shading in building1.idfobjects["WindowShadingControl"]:
    print(shading)
for ventilation in building1.idfobjects["ZoneVentilation:DesignFlowRate"]:
    print(ventilation)
for material in building1.idfobjects["Material"]:
    print(material)
'''


WindowShadingControl,
    1001-Block1:Zone3,        !- Name
    Block1:Zone3,             !- Zone Name
    1,                        !- Shading Control Sequence Number
    ExteriorBlind,            !- Shading Type
    2001,                     !- Construction with Shading Name
    OnIfHighZoneAirTemperature,    !- Shading Control Type
    TM59_Kitchen_Occ,         !- Schedule Name
    24,                       !- Setpoint
    Yes,                      !- Shading Control Is Scheduled
    No,                       !- Glare Control Is Active
    ,                         !- Shading Device Material Name
    FixedSlatAngle,           !- Type of Slat Angle Control for Blinds
    ,                         !- Slat Angle Schedule Name
    0,                        !- Setpoint 2
    Block1:Zone3,             !- Daylighting Control Object Name
    Sequential,               !- Multiple Surface Control Type
    Block1:Zone3_Wall_5_0_0_0_0_0_Win;    !- Fenestration Surface 1 Name


WindowShadingCon

In [9]:
## Set up evaluator
# Parameter Space
parameters = []

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientation_descriptor = CategoryParameter(options=[0, 90, 180, 270])

insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792', 
                           field_name='Thickness')
insulation_descriptor = RangeParameter(min_val = 0.01, max_val=0.99) 

shading = FieldSelector(class_name='WindowShadingControl',field_name='Setpoint')
shading_descriptor = RangeParameter(min_val = 0, mtickness_values = np.linspace(0.001,0.35,10)ax_val=30) 

ventilation = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',field_name='Design Flow Rate')
ventilation_descriptor = CategoryParameter(options=[0, 0.1])


parameters.append(wwr(RangeParameter(0.1,0.9))) 

parameters.append(Parameter(selector=orientation, 
                              value_descriptor=orientation_descriptor,
                              name='Orientation'))

parameters.append(Parameter(selector=insulation,
                            value_descriptor=insulation_descriptor,
                            name='Insulation Thickness'))

parameters.append(Parameter(selector=shading,
                            value_descriptor=shading_descriptor,
                            name='Shading'))


parameters.append(Parameter(selector=ventilation,
                            value_descriptor=ventilation_descriptor,
                            name='Ventilation'))


# Ask prof about shading and ventilation if they are ok and Uvalue calculation
print(parameters)

[Parameter(name='Window to Wall Ratio', selector=wwrSelector(), value_descriptor=RangeParameter(min=0.1, max=0.9)), Parameter(name='Orientation', selector=FieldSelector(field_name='North Axis', class_name='Building'), value_descriptor=CategoryParameter(options=[0, 90, 180, 270])), Parameter(name='Insulation Thickness', selector=FieldSelector(field_name='Thickness', class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792'), value_descriptor=RangeParameter(min=0.01, max=0.99)), Parameter(name='Shading', selector=FieldSelector(field_name='Setpoint', class_name='WindowShadingControl'), value_descriptor=RangeParameter(min=0, max=30)), Parameter(name='Ventilation', selector=FieldSelector(field_name='Design Flow Rate', class_name='ZoneVentilation:DesignFlowRate'), value_descriptor=CategoryParameter(options=[0, 0.1]))]


In [13]:
# Objectives of our problem which is to minimize the TOTAL COMSUMPTION
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']

# problem = parameters + objectives
besos_problem = EPProblem(parameters, objectives)

# EPProblem vs EvaluatorEP = problem + building + weather
evaluator = EvaluatorEP(besos_problem, building, out_dir='outputdir', err_dir='outputdir', epw_file=weather_file) 

# evaluate the problem with some possible values
samples = sampling.dist_sampler(sampling.lhs, besos_problem, num_samples=1)


# result
outputs = evaluator.df_apply(samples, keep_input=True)
outputs.describe()
type(outputs)

'''
result = evaluator([0, 0.5, 0.0792]) 
values = dict(zip(objectives, result))
for key, value in values.items():
    print(key, " :: ", "{0:.2f}".format(value/3.6e6), "kWh")
'''



'\nresult = evaluator([0, 0.5, 0.0792]) \nvalues = dict(zip(objectives, result))\nfor key, value in values.items():\n    print(key, " :: ", "{0:.2f}".format(value/3.6e6), "kWh")\n'

In [25]:
# Q is kWh / m2
total_area= 0
for Zones in building1.idfobjects["Zone"]:
    total_area += float(Zones.checkrange("Floor_Area")) 

In [14]:
outputs

,Window to Wall Ratio,Orientation,Insulation Thickness,Shading,Ventilation,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility
0,0.114066,180,0.149682,0.39548,0.1,5.773034e+09,0.0,1.553372e+10


In [26]:
outputs['total (Q)'] = outputs[objectives].sum(axis=1) /3.6e6 /total_area 
outputs

,Window to Wall Ratio,Orientation,Insulation Thickness,Shading,Ventilation,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility,total,total (Q)
0,0.114066,180,0.149682,0.39548,0.1,5.773034e+09,0.0,1.553372e+10,2.130676e+10,126.29244
